In [1]:
def distv3d(halo, center):
    norm = np.sqrt(np.square(center['vx'] - halo['vx']) + 
                   np.square(center['vy'] - halo['vy']) + 
                   np.square(center['vz'] - halo['vz']))
    return norm

def distv(halo, center):
    norm = center['vx'] - halo['vx'] + \
           center['vy'] - halo['vy'] + \
           center['vz'] - halo['vz']
    return norm


def dist(halo, center):
    norm = np.sqrt(np.square(center['x'] - halo['x']) + 
                   np.square(center['y'] - halo['y']) + 
                   np.square(center['z'] - halo['z']))
    return norm 

def match_gal_hal_tree(gt, ht):
    nout = 187
    dt = 3 # compare progenitor at dt ago.
    
    gal_now = gt[gt["nout"]==nout]
    hal_now = ht[ht["nout"]==nout]
    
    gal_before = gt[gt["nout"]==nout-dt]
    hal_before = ht[ht["nout"]==nout-dt]    
    
    dominant = 0.1 # matched one should have less error by this amount or smaller 
                    # compared to the second best matched one.
    
    abs_tol_pos = 5e-5 # Position absolute tolerance [in code unit?]
    abs_tol_vel = 10   # velocity absolute tolerance [in kms?]
    
    for gal in gal_now:
        dd = dist(hal_now, gal)
        vv = distv(hal_now, gal)
        d_sort = np.argsort(dd)
        v_sort = np.argsort(vv)
        if (dd[d_sort[0]] < dominant * dd[d_sort[1]]) and (dd[d_sort[0]] < abs_tol_pos) and \
        (vv[v_sort[0]] < dominant * vv[v_sort[1]]) and (vv[v_sort[0]] < abs_tol_vel):
            gal['hosthalo'] = allhal.data['id'][d_sort[0]]
            i0.append(i)
            newhals[i] = allhal.data[d_sort[0]]
        else:
            atree = tree.atree(gt)
            prg = atree[dt]
            for gal2 in gal_before:
                dd = dist(hal_now, gal2)
                vv = distv(hal_now, gal2)
                d_sort = np.argsort(dd)
                v_sort = np.argsort(vv)
            
                

def get_comp_dist(hal_now, gal, nreturn=5):
    """Measure 6D distance and return N closest entries"""
    dd = dist(hal_now, gal)
    vv = distv(hal_now, gal)
    dd_q1 = np.percentile(dd,10)
    vv_q1 = np.percentile(vv,10)
    comp_dist = np.sqrt(np.square(dd/dd_q1) + np.square(vv/vv_q1))
    ind_sort = np.argsort(comp_dist)
    return comp_dist[ind_sort[:nreturn]], hal_now[ind_sort[:nreturn]]

def before_to_now(htdata, hals, dt):
    out = []
    for hal in hals:
        atree_hal = ctu.extract_main_tree_full(htdata, idx=hal['id'])
        out.append(atree_hal[hal['nout'] + dt])
    return out

def now_to_before(htdata, hals, dt):
    """
    progenitor of current halos.
    If does not exist, give -1
    """
    out =[]
    for hal in hals:
        idx = hal['id']
        try:
            #smalldata = htdata[htdata['tree_root_id'] == idx]
            #print(smalldata)
            atree_hal = ctu.extract_main_tree(htdata, idx=idx)
            out.append(atree_hal[dt])
        except:
            print("broken tree")
            out.append(-1)        
            
    return np.array(out)

In [145]:
import load
import draw
from tree import ctutils as ctu
from tree import halomodule as hmo
from load.info import Info
import pandas as pd
import pickle
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import tree.treemodule as tmo
from load.rd_GM import Gal

# Load galaxy catalog
# and halo catalog
clusters = ["29176"]
nout_fi = 187

#dominant = 0.4 # matched one should have less error by this amount or smaller 
                # compared to the second best matched one.

#abs_tol_pos = 1e-2 # Position absolute tolerance [in Mpc]
#abs_tol_vel = 100   # velocity absolute tolerance [in kms?]

dts = [3,5]

mstar_all = []
mhal_all = []

#for cluster in clusters:
cluster = clusters[0]
wdir = './' + cluster + '/'
gt = tmo.load_tree(wdir, is_gal=True)
ht = tmo.load_tree(wdir, is_gal=False)

gt.data["rvir"] *=1e-3
ht.data["rvir"] *=1e-3

FileNotFoundError: [Errno 2] No such file or directory: './29176/GalaxyMaker/Trees/tree_0_0_0.dat'

In [3]:
s = load.sim.Sim(base=wdir,nout=187)

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.


In [4]:
s.add_part(ptypes=["star id pos mass", "dm id pos mass"])

Types of particles you want to load are:  ['star id pos mass', 'dm id pos mass']
[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Part
A particle instance is created

Loading by fortran module
Fortran-reading done


In [5]:
nout_fi = 187
info = Info(nout_fi, base=wdir)

gal_now = gt.data[gt.data["nout"]==nout_fi]
hal_now = ht.data[ht.data["nout"]==nout_fi]


In [6]:
len(gal_now)

125

In [7]:
# Only long enough trees
gal_ok = ctu.check_tree_complete(gt.data,
                                 nout_fi - max(dts),
                                 nout_fi, gal_now["id"],
                                 idx=True)

In [8]:
print("{} ({}%) out of {} galaxies are long enough.".format(
    len(gal_ok), 100*len(gal_ok)/len(gal_now), len(gal_now)))

125 (100.0%) out of 125 galaxies are long enough.


In [9]:
# Test -3, -5 snapshot halos

hal_3 = ht.data[ht.data["nout"]==nout_fi - 3]
hal_5 = ht.data[ht.data["nout"]==nout_fi - 5]
hal_this_list = [hal_3, hal_5]

In [10]:
    comp_dists=[]
    result = []
    mhal_result = []
    dist_error = []

In [11]:
gal = gal_now[0]

In [12]:
from scipy.spatial import cKDTree

def get_kd_matched_hals(kdtree, gal, n_match=5, rscale = 2.0):
    #len_tree = kdtree.length
    dd, ind = kdtree.query((gal["x"],
                            gal["y"],
                            gal["z"]),
                            distance_upper_bound=gal["rvir"] * rscale,
                            k=n_match)
    
    return dd[np.isfinite(dd)], ind[np.isfinite(dd)]

# KDTree
kdtree = cKDTree(np.stack((hal_now["x"],
                           hal_now["y"],
                           hal_now["z"]),
                          axis=1))

In [110]:
dd, inds = get_kd_matched_hals(kdtree, gal, n_match=20, rscale = 1.0)

In [111]:
fig, ax = plt.subplots()

draw.pp.pp_halo(hal_now[inds], npix=800, ax=ax, radius="rvir", edgecolor="red")
draw.pp.pp_halo(gal_now, npix=800, ax=ax, radius="rvir",
                edgecolor="blue")
draw.pp.update_tick_labels(ax)
new_region = region_gm2code(ax.pp_hal_meta.region, s.info)

plt.show()

In [81]:
from matplotlib.colors import LogNorm

In [95]:
nxc = new_region["xc"]
nyc = new_region["yc"]
nrad = new_region["radius"]

In [133]:
gcat = hmo.Halo(base=wdir, nout=187, is_gal=True)

[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =./29172/


In [104]:
fig, ax = plt.subplots()
ax.hist2d(s.part.star["x"]*s.info.cboxsize,
           s.part.star["y"]*s.info.cboxsize,
           range = [hal_region["xr"], hal_region["yr"]],
           #range=[[nxc - 0.2*nrad, nxc+0.2*nrad],
           #       [nyc - 0.2*nrad, nyc+0.2*nrad]],
           bins=800, norm=LogNorm())
plt.show()

/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/matplotlib/colors.py:930: RuntimeWarning: divide by zero encountered in log
  resdat -= np.log(vmin)
/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/matplotlib/colors.py:931: RuntimeWarning: divide by zero encountered in log
  resdat /= (np.log(vmax) - np.log(vmin))
/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/matplotlib/colors.py:931: RuntimeWarning: invalid value encountered in true_divide
  resdat /= (np.log(vmax) - np.log(vmin))


In [61]:
def region_gm2code(hal_region, info):
    """
    Most of the time it is more convenient to work in GM Kpc unit, 
    but I don't want to convert all stellar/DM particle units. 
    """
    import utils.sampling as smp
    
    New_region = hal_region.copy()
    return smp.set_region(radius=hal_region["radius"]/info.cboxsize,
                          centers=np.array(hal_region["centers"])/info.cboxsize)

In [66]:
# stellar map + halo
smap= draw.pp.part2den(s.part.star, s.info, region=new_region)


x.ptp() 800.0 npix 800
Resampling 1903768 values to a 800 by 800 by 1 grid


In [143]:
gcat.data["x"] *= s.info.cboxsize
gcat.data["y"] *= s.info.cboxsize
gcat.data["z"] *= s.info.cboxsize
gcat.data["rvir"] *= s.info.cboxsize

In [144]:
fig, ax = plt.subplots()

#ax.hist2d
draw.pp.pp_halo(hal_now,region=hal_region, npix=800, ax=ax, radius="rvir", edgecolor="red")
histmap = np.histogram2d(s.part.star["x"]*s.info.cboxsize,
           s.part.star["y"]*s.info.cboxsize,
           range = [hal_region["xr"], hal_region["yr"]],
           #range=[[nxc - 0.2*nrad, nxc+0.2*nrad],
           #       [nyc - 0.2*nrad, nyc+0.2*nrad]],
           bins=800)#, norm=LogNorm())
ax.imshow(np.log10(histmap[0]).T, origin="lower")
#ax.pp_hal_meta.region=hal_region
#draw.pp.pp_halo(hal_now, npix=800, ax=ax, radius="rvir", edgecolor="red")
draw.pp.pp_halo(hal_now, npix=800, ax=ax, radius="rvir", edgecolor="red")
#draw.pp.pp_halo(gal_now, npix=800, ax=ax, radius="rvir",edgecolor="blue")
draw.pp.pp_halo(gcat, npix=800, ax=ax, radius="rvir",edgecolor="blue")
#new_region = region_gm2code(axs[0].pp_hal_meta.region, s.info)

plt.show()

/home/hoseung/anaconda3/envs/hspy/lib/python3.6/site-packages/ipykernel/__main__.py:11: RuntimeWarning: divide by zero encountered in log10


In [117]:
plt.close()

In [58]:
plt.hist(s.part.star["x"])
plt.show()

In [37]:
draw.pp.pp_halo(gt.data[gt.data["nout"]==187], npix=400, ax=axs[0], radius="rvir",
                edgecolor="green")
draw.pp.pp_halo(good_gals, npix=400, ax=axs[0], radius="rvir", edgecolor="red")
draw.pp.update_tick_labels(axs[0])


# First, fix the region
draw.pp.pp_halo(good_gals, npix=400, ax=axs[1], radius="rvir", edgecolor="red", proj="x",
                name=True)
draw.pp.pp_halo(ht.data[ht.data["nout"]==187], npix=400, ax=axs[1], radius="rvir",
                edgecolor="blue", proj="x", verbose=True)
draw.pp.pp_halo(gt.data[gt.data["nout"]==187], npix=400, ax=axs[1], radius="rvir",
                edgecolor="green", proj="x")
# Second, overplot.
draw.pp.pp_halo(good_gals, npix=400, ax=axs[1], radius="rvir", edgecolor="red", proj="x",
                name=True)


draw.pp.update_tick_labels(axs[1])

plt.show()

NameError: name 'good_gals' is not defined

In [36]:
dd

array([ 0.04082196,  0.09922437,  0.10525189,  0.12127102,  0.15133261])

In [18]:
# Halo descendant - galaxy descendant
comp_dist, good_hals_now = get_comp_dist(hal_now, gal, nreturn=5)
# halo must be more massive than the galaxy
matches=[]
good_hals_now = good_hals_now[good_hals_now["m"] > gal["m"]]
#print("M_g",gal["m"], "ratio", gal["m"]/good_hals_now["mvir"])
atree = ctu.extract_main_tree(gt.data, idx=gal['id'])

matches.append(good_hals_now["Orig_halo_id"])

In [23]:
dd = dist(hal_now, gal)

In [25]:
plt.hist(np.log10(dd))
plt.show()

In [19]:
comp_dist

array([ 0.22061806,  0.31734417,  0.32389218,  0.33239106,  0.39408722])